### srt to txt

In [7]:
import pysrt
import os 
import string
from os import path
import enchant
from guess_language import guess_language
from itertools import compress
import time 
from datetime import datetime
import pandas as pd

## Define some functions first 

In [8]:
##############################
#### define some functions ###
##############################
def check(f):
    filename, file_extension = os.path.splitext(f)
    if file_extension == '.srt':
        return True
    else:
        return False

def check_alignment(ele):
    t2 = ele.text.split('\n')[0]  ## get the first element, see if it is a time stamp
    if '-->' in t2:
        t1_end = ele.end.seconds
        start,end = t2.split(' --> ')
        end = time.strptime(end,'%H:%M:%S,%f')
        t2_end = end.tm_sec
        #print(t1_end," -- ",t2_end)
        if abs(t1_end - t2_end) > 3:
            return False
        else:
            return True
    else:
        return True
    
def get_text_line(ele):
    if check_alignment(ele) is False:
        return False
    text_list = ele.text.split('\n')
    languages = [guess_language(l) for l in text_list]
    en_index = [l=='en' for l in languages]
    zh_index = [l=='zh' for l in languages]
    en_lines = list(compress(text_list,en_index))
    zh_lines = list(compress(text_list,zh_index))
    if len(en_lines)==0 or len(zh_lines)==0:
        return None
    en_text = ' '.join(en_lines)
    zh_text = ' '.join(zh_lines)
    text_line = en_text + ' | ' + zh_text
    return text_line

def get_doc_txt(subs):
    doc = []
    for ele in subs:
        clean_text = get_text_line(ele)
        if clean_text == False:
            return None
        elif clean_text is not None:
            doc.append(clean_text)
    return doc 

def check_last_line(doc):
    en,zh = doc[-1].split('|')
    return len(en)/len(zh)

In [9]:
data_folder = 'data/cn/data/'
result_folder = 'data/cn/result_folder/'

In [10]:
## check if extension is .srt
srts = [data_folder + f for f in os.listdir(data_folder) if check(f)]
len(srts)

2210

In [ ]:
srts[0]

'data/cn/data/乔治·帕潘德里欧: 设想一个跨国界的欧洲民主.srt'

In [ ]:
doc_list ={'title':[],'last':[],'status':[]}

for index,srt in enumerate(srts):
    print (len(srts),index+1)
    subs = pysrt.open(srt)
    doc = get_doc_txt(subs)
    fname = srt.split('/')[-1].split('.')[-2]+ '.txt'
    doc_list['title'].append(fname)
    if doc is None:
        doc_list['last'].append('Failed')
        doc_list['status'].append('Failed')
    else:
        with open(result_folder + fname, mode='wt', encoding='utf-8') as myfile:
            myfile.write('\n'.join(doc))
        doc_list['last'].append(check_last_line(doc))
        doc_list['status'].append('Success')

2210 1
2210 2
2210 3
2210 4
2210 5
2210 6
2210 7
2210 8
2210 9
2210 10
2210 11
2210 12
2210 13
2210 14
2210 15
2210 16
2210 17
2210 18
2210 19
2210 20
2210 21
2210 22
2210 23
2210 24
2210 25
2210 26
2210 27
2210 28
2210 29
2210 30
2210 31
2210 32
2210 33
2210 34
2210 35
2210 36
2210 37
2210 38
2210 39
2210 40
2210 41
2210 42
2210 43
2210 44
2210 45
2210 46
2210 47
2210 48
2210 49
2210 50
2210 51
2210 52
2210 53
2210 54
2210 55
2210 56
2210 57
2210 58
2210 59
2210 60
2210 61
2210 62
2210 63
2210 64
2210 65
2210 66
2210 67
2210 68
2210 69
2210 70
2210 71
2210 72
2210 73
2210 74
2210 75
2210 76
2210 77
2210 78
2210 79
2210 80
2210 81
2210 82
2210 83
2210 84
2210 85
2210 86
2210 87
2210 88
2210 89
2210 90
2210 91
2210 92
2210 93
2210 94
2210 95
2210 96
2210 97
2210 98
2210 99
2210 100
2210 101
2210 102
2210 103
2210 104
2210 105
2210 106
2210 107
2210 108
2210 109
2210 110
2210 111
2210 112
2210 113
2210 114
2210 115
2210 116
2210 117
2210 118
2210 119
2210 120
2210 121
2210 122
2210 123
2

In [ ]:
## export log file to csv
df = pd.DataFrame(doc_list)
df.to_csv('ted_txt.csv')

In [ ]:
df_problem=df[df.status!='Failed']
df_problem['last'] = df_problem['last'].astype(float)
df_problem[(df_problem['last']>5) | (df_problem['last']<1)]

In [ ]:
######################################################
### no longer use this, used guess_language instead###
### this is for future reference                   ###
######################################################

# ### ehck each line, see if they are time header, or english or chinese
# def is_english(word):
#     try:
#         return word.encode('ascii').isalpha()
#         en = [e for e in word if word.encode('ascii').isalpha()]
#     except:
#         return False
    
# def is_chinese(word):
#     ch = [c for c in word if u'\u4e00' <= c <= u'\u9fff']
#     if len(ch) > 0:
#         return True
#     else:
#         return False

# def check_language(line):
#     if '-->' in line:
#         return 'tm'  ## time header
#     elif is_english(line):
#         return 'en'
#     elif is_chinese(line):
#         return 'cn'
#     else:
#         return None